In [195]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [121]:
seinfeld = pd.read_csv('seinfeld_episode_rankings.csv')

In [122]:
seinfeld

,Title,Plot,Quotability/cultural impact,Laughs,Total score,IMDb rating,IMDb rank,Notes,Season,episode
0,The Seinfeld Chronicles 1:1,4.0,7.0,7.0,18.0,7.6,76.0,2nd button joke,1.0,1.0
1,Male Unbonding 1:2,3.0,6.5,2.0,11.5,7.3,83.0,Kramer's pizza idea,1.0,2.0
2,The Stakeout 1:3,4.0,7.0,9.0,20.0,7.7,72.0,Art Vandelay first appearance,1.0,3.0
3,The Robbery 1:4,3.0,3.5,5.5,12.0,7.6,76.0,NaN,1.0,4.0
4,The Stock Tip 1:5,3.0,3.0,6.5,12.5,7.5,78.0,NaN,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...
174,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
175,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
176,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
177,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [123]:
seinfeld.columns=['title','plot','quotability','laughs','total_score','imdb_rating','imdb_rank','notes','season','episode']

In [124]:
seinfeld=seinfeld[0:68]

In [125]:
seinfeld.sort_values('total_score',ascending=False)

,title,plot,quotability,laughs,total_score,imdb_rating,imdb_rank,notes,season,episode
50,The Contest 4:11,5.0,10.0,31.0,46.0,9.6,1.0,first appearance of Estelle.,4.0,11.0
66,The Sniffing Accountant,4.0,10.0,27.5,41.5,8.0,60.0,"""heres to feeling good all the time""",5.0,4.0
59,The Junior Mint,4.0,10.0,27.0,41.0,8.7,11.0,"""DOLORES!!!""",4.0,20.0
52,The Pick 4:13,4.0,9.0,24.0,37.0,8.5,21.0,NaN,4.0,13.0
63,The Mango,4.0,7.0,25.5,36.5,8.4,24.0,"""you faked it?""",5.0,1.0
...,...,...,...,...,...,...,...,...,...,...
20,The Dog 3:4,3.0,3.0,7.0,13.0,7.3,83.0,NaN,3.0,4.0
14,The Baby Shower 2:10,3.0,3.0,7.0,13.0,7.5,78.0,NaN,2.0,10.0
4,The Stock Tip 1:5,3.0,3.0,6.5,12.5,7.5,78.0,NaN,1.0,5.0
3,The Robbery 1:4,3.0,3.5,5.5,12.0,7.6,76.0,NaN,1.0,4.0


In [126]:
scripts = pd.read_csv('seinfeld_data/scripts.csv')

In [127]:
episodes = pd.read_csv('seinfeld_data/episode_info.csv')

In [128]:
episodes = episodes.drop(columns='Unnamed: 0')

In [129]:
episodes.Title = episodes.Title.astype(str)

In [130]:
episodes.Title = episodes.Title.str.lower().str.replace(' ','_').str.replace(',','')

In [131]:
scripts = scripts.drop(columns='Unnamed: 0')

In [132]:
scripts[0:211].SEID = 'S01E00'
scripts[0:211].EpisodeNo = 0.0

In [133]:
episodes.SEID[0] = 'S01E00'
episodes.EpisodeNo[0] = 0.0

In [134]:
scripts

,Character,Dialogue,EpisodeNo,SEID,Season
0,JERRY,Do you know what this is all about? Do you kno...,0.0,S01E00,1.0
1,JERRY,"(pointing at Georges shirt) See, to me, that b...",0.0,S01E00,1.0
2,GEORGE,Are you through?,0.0,S01E00,1.0
3,JERRY,"You do of course try on, when you buy?",0.0,S01E00,1.0
4,GEORGE,"Yes, it was purple, I liked it, I dont actuall...",0.0,S01E00,1.0
...,...,...,...,...,...
54611,JERRY,Grand theft auto - don't steal any of my jokes.,23.0,S09E23,9.0
54612,PRISONER 3,You suck - I'm gonna cut you.,23.0,S09E23,9.0
54613,JERRY,"Hey, I don't come down to where you work, and ...",23.0,S09E23,9.0
54614,GUARD,"Alright, Seinfeld, that's it. Let's go. Come on.",23.0,S09E23,9.0


In [135]:
scripts[scripts.SEID=='S01E00']

,Character,Dialogue,EpisodeNo,SEID,Season
0,JERRY,Do you know what this is all about? Do you kno...,0.0,S01E00,1.0
1,JERRY,"(pointing at Georges shirt) See, to me, that b...",0.0,S01E00,1.0
2,GEORGE,Are you through?,0.0,S01E00,1.0
3,JERRY,"You do of course try on, when you buy?",0.0,S01E00,1.0
4,GEORGE,"Yes, it was purple, I liked it, I dont actuall...",0.0,S01E00,1.0
...,...,...,...,...,...
206,LAURA,"Yeah, yeah, hes a great guy...",0.0,S01E00,1.0
207,JERRY,Yeah.,0.0,S01E00,1.0
208,LAURA,"Youd really like him, you know, I cant wait to...",0.0,S01E00,1.0
209,JERRY,Me too!,0.0,S01E00,1.0


In [209]:
scripts.Character=scripts.Character.str.replace(r"\(.*\)","")

In [210]:
line_count=scripts.Character.value_counts().head(475)

In [217]:
scripts.Character.str.strip().value_counts()

JERRY                                                                                               14905
GEORGE                                                                                               9781
ELAINE                                                                                               8073
KRAMER                                                                                               6723
NEWMAN                                                                                                649
                                                                                                    ...  
(ELAINE finally notices them, but suddenly from the other direction, three other guys are coming        1
ALL THREE                                                                                               1
JERR                                                                                                    1
Dx                                            

In [138]:
episodes["SEID"] = episodes.SEID.astype(str)
scripts["SEID"] = scripts.SEID.astype(str)

In [139]:
scripts[scripts.SEID=='S01E03']

,Character,Dialogue,EpisodeNo,SEID,Season
837,JERRY,"Most men like working on things. Tools, object...",3.0,S01E03,1.0
838,GEORGE,I had to say something. (chuckles) I had to sa...,3.0,S01E03,1.0
839,JERRY,I dont think you did anything wrong.,3.0,S01E03,1.0
840,GEORGE,I told her I liked her. Why? Why did I tell he...,3.0,S01E03,1.0
841,JERRY,We can only thank God for that.,3.0,S01E03,1.0
...,...,...,...,...,...
1073,JOEL,"You know, I should really try something like t...",3.0,S01E03,1.0
1074,JERRY,You really should.,3.0,S01E03,1.0
1075,JOEL,"Well, lets just take a look here.",3.0,S01E03,1.0
1076,JOEL,Forty-one home games. Let's see saturday night...,3.0,S01E03,1.0


In [140]:
pd.merge(episodes, right=scripts, on='SEID')

,Season_x,EpisodeNo_x,Title,AirDate,Writers,Director,SEID,Character,Dialogue,EpisodeNo_y,Season_y
0,1.0,0.0,good_news_bad_news,"July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E00,JERRY,Do you know what this is all about? Do you kno...,0.0,1.0
1,1.0,0.0,good_news_bad_news,"July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E00,JERRY,"(pointing at Georges shirt) See, to me, that b...",0.0,1.0
2,1.0,0.0,good_news_bad_news,"July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E00,GEORGE,Are you through?,0.0,1.0
3,1.0,0.0,good_news_bad_news,"July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E00,JERRY,"You do of course try on, when you buy?",0.0,1.0
4,1.0,0.0,good_news_bad_news,"July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E00,GEORGE,"Yes, it was purple, I liked it, I dont actuall...",0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
54611,9.0,23.0,the_finale,"May 14, 1998",Larry David,Andy Ackerman,S09E23,JERRY,Grand theft auto - don't steal any of my jokes.,23.0,9.0
54612,9.0,23.0,the_finale,"May 14, 1998",Larry David,Andy Ackerman,S09E23,PRISONER 3,You suck - I'm gonna cut you.,23.0,9.0
54613,9.0,23.0,the_finale,"May 14, 1998",Larry David,Andy Ackerman,S09E23,JERRY,"Hey, I don't come down to where you work, and ...",23.0,9.0
54614,9.0,23.0,the_finale,"May 14, 1998",Larry David,Andy Ackerman,S09E23,GUARD,"Alright, Seinfeld, that's it. Let's go. Come on.",23.0,9.0
